# Transfer Learning : Model Selection

In [18]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.utils.np_utils import to_categorical
import keras.backend as K
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

In [19]:
train_data_dir = 'Data/Training_Data'
validation_data_dir = 'Data/Validation_Data'
test_data_dir= 'Data/Test_Data'

train_labels_file = 'Labels/training_labels.npy'
validation_labels_file = 'Labels/validation_labels.npy'
test_labels_file = 'Labels/test_labels.npy'

img_width, img_height = 224, 224
NB_CLASSES = 10
epochs = 16
batch_size = 512

In [20]:
train_labels = np.load(open(train_labels_file, 'rb'))
validation_labels = np.load(open(validation_labels_file, 'rb'))
test_labels = np.load(open(test_labels_file, 'rb'))

## Converting images to feature vectors using weights from ImageNet

In [21]:
def images_to_feature_vectors(model, directory, batch_size, steps):
    
    datagen = ImageDataGenerator()
    generator = datagen.flow_from_directory(
        directory,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) # Keep the data in the same order
    
    features = model.predict_generator(generator, steps, verbose=1) 
    
    return features

## Train the top part of the model

In [22]:
def top_model(training_features, validation_features, batch_size, epochs):
    model = Sequential()
    model.add(Flatten(input_shape=training_features.shape[1:], name='Main_input'))
    model.add(BatchNormalization())
    model.add(Dense(1096, activation='relu', name='D1'))
    model.add(Dense(512, activation='relu', name='D2'))
    model.add(Dense(NB_CLASSES, activation='softmax', name='Main_output'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])
    train_labels_onehot = to_categorical(train_labels, NB_CLASSES)            #One Hot Encoder
    validation_labels_onehot = to_categorical(validation_labels, NB_CLASSES)  #One Hot Encoder
    
    model.fit(training_features, train_labels_onehot,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_features, validation_labels_onehot))
   
    return model

## VGG-16 Evaluation

In [23]:
start_time = datetime.now()

# Batch size has to be a multiple of the number of images  to keep our vectors consistents
training_batch_size = 1 # batch size for feature pre-training
validation_batch_size = 1 # batch size for feature pre-training

model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3)) #VGG16 trained on imagenet
training_features = images_to_feature_vectors(model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

end_time = datetime.now()
features_extraction_time = end_time - start_time
print('Features extraction time : {}'.format(features_extraction_time))

Found 307 images belonging to 11 classes.
307/307 [==============================] - 78s 253ms/step
Found 76 images belonging to 11 classes.
76/76 [==============================] - 20s 259ms/step
Features extraction time : 0:01:38.691649


In [25]:
start_time = datetime.now()

model = top_model(training_features, validation_features, batch_size, epochs)

end_time = datetime.now()
training_time = end_time - start_time
print('Total training duration : {}'.format(training_time))

Instructions for updating:
Use tf.cast instead.
Train on 307 samples, validate on 76 samples
Epoch 1/16
307/307 [==============================] - 3s 11ms/step - loss: 2.5939 - acc: 0.0814 - val_loss: 1.9731 - val_acc: 0.7895
Epoch 2/16
307/307 [==============================] - 1s 4ms/step - loss: 0.0073 - acc: 0.9935 - val_loss: 2.3592 - val_acc: 0.7763
Epoch 3/16
307/307 [==============================] - 1s 4ms/step - loss: 6.4147e-04 - acc: 1.0000 - val_loss: 2.6924 - val_acc: 0.7632
Epoch 4/16
307/307 [==============================] - 1s 4ms/step - loss: 0.0035 - acc: 1.0000 - val_loss: 2.6963 - val_acc: 0.7763
Epoch 5/16
307/307 [==============================] - 1s 4ms/step - loss: 0.0015 - acc: 1.0000 - val_loss: 2.6011 - val_acc: 0.7763
Epoch 6/16
307/307 [==============================] - 1s 4ms/step - loss: 2.3079e-04 - acc: 1.0000 - val_loss: 2.5138 - val_acc: 0.7895
Epoch 7/16
307/307 [==============================] - 1s 4ms/step - loss: 1.6356e-04 - acc: 1.0000 - val_l

In [26]:
train_labels_onehot = to_categorical(train_labels, NB_CLASSES)
loss, training_accuracy = model.evaluate(training_features,train_labels_onehot)

307/307 [==============================] - 0s 1ms/step


In [27]:
validation_labels_onehot = to_categorical(validation_labels, NB_CLASSES)
loss, validation_accuracy = model.evaluate(validation_features,validation_labels_onehot)

76/76 [==============================] - 0s 1ms/step


In [28]:
'Training Accuracy : '+ str(round(training_accuracy*100,1)) + '%'

'Training Accuracy : 100.0%'

In [29]:
'Validation Accuracy : '+ str(round(validation_accuracy*100,1)) + '%'

'Validation Accuracy : 80.3%'

In [30]:
vgg16_results = (round(training_accuracy*100,1), round(validation_accuracy*100,1), round(features_extraction_time.total_seconds()), round(training_time.total_seconds()))

## Inception Evaluation

In [31]:
start_time = datetime.now()

model = applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3)) #VGG16 trained on imagenet
training_features = images_to_feature_vectors(model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

end_time = datetime.now()
features_extraction_time = end_time - start_time
print('Features extraction time : {}'.format(features_extraction_time))

Found 307 images belonging to 11 classes.
307/307 [==============================] - 47s 153ms/step
Found 76 images belonging to 11 classes.
76/76 [==============================] - 11s 147ms/step
Features extraction time : 0:01:18.440715


In [32]:
start_time = datetime.now()

model = top_model(training_features, validation_features, batch_size, epochs)

end_time = datetime.now()
training_time = end_time - start_time
print('Total training duration : {}'.format(training_time))

Train on 307 samples, validate on 76 samples
Epoch 1/16
307/307 [==============================] - 7s 23ms/step - loss: 2.7096 - acc: 0.0847 - val_loss: 8.6085 - val_acc: 0.3816
Epoch 2/16
307/307 [==============================] - 2s 8ms/step - loss: 4.8481 - acc: 0.6091 - val_loss: 8.7810 - val_acc: 0.4079
Epoch 3/16
307/307 [==============================] - 2s 7ms/step - loss: 3.8912 - acc: 0.7101 - val_loss: 9.0170 - val_acc: 0.4079
Epoch 4/16
307/307 [==============================] - 2s 8ms/step - loss: 3.6917 - acc: 0.7101 - val_loss: 8.6848 - val_acc: 0.3947
Epoch 5/16
307/307 [==============================] - 2s 8ms/step - loss: 3.5178 - acc: 0.7557 - val_loss: 8.4848 - val_acc: 0.4211
Epoch 6/16
307/307 [==============================] - 2s 8ms/step - loss: 3.3727 - acc: 0.7655 - val_loss: 7.8308 - val_acc: 0.4211
Epoch 7/16
307/307 [==============================] - 2s 8ms/step - loss: 3.3574 - acc: 0.7622 - val_loss: 7.8906 - val_acc: 0.4605
Epoch 8/16
307/307 [==========

In [33]:
loss, training_accuracy = model.evaluate(training_features,train_labels_onehot)

307/307 [==============================] - 1s 2ms/step


In [34]:
loss, validation_accuracy = model.evaluate(validation_features,validation_labels_onehot)

76/76 [==============================] - 0s 2ms/step


In [35]:
'Training Accuracy : '+ str(round(training_accuracy*100,1)) + '%'

'Training Accuracy : 88.9%'

In [36]:
'Validation Accuracy : '+ str(round(validation_accuracy*100,1)) + '%'

'Validation Accuracy : 59.2%'

In [37]:
inception_results = (round(training_accuracy*100,1), round(validation_accuracy*100,1), round(features_extraction_time.total_seconds()), round(training_time.total_seconds()))

## ResNet Evaluation

In [38]:
start_time = datetime.now()

model = applications.ResNet50(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3)) #VGG16 trained on imagenet
training_features = images_to_feature_vectors(model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

end_time = datetime.now()
features_extraction_time = end_time - start_time
print('Features extraction time : {}'.format(features_extraction_time))

Found 307 images belonging to 11 classes.
307/307 [==============================] - 61s 197ms/step
Found 76 images belonging to 11 classes.
76/76 [==============================] - 14s 187ms/step
Features extraction time : 0:01:30.207649


In [39]:
start_time = datetime.now()

model = top_model(training_features, validation_features, batch_size, epochs)

end_time = datetime.now()
training_time = end_time - start_time
print('Total training duration : {}'.format(training_time))

Train on 307 samples, validate on 76 samples
Epoch 1/16
307/307 [==============================] - 13s 43ms/step - loss: 2.8549 - acc: 0.1107 - val_loss: 1.4155 - val_acc: 0.8684
Epoch 2/16
307/307 [==============================] - 5s 15ms/step - loss: 2.8593e-06 - acc: 1.0000 - val_loss: 2.2279 - val_acc: 0.8158
Epoch 3/16
307/307 [==============================] - 5s 15ms/step - loss: 0.0597 - acc: 0.9935 - val_loss: 3.0368 - val_acc: 0.7895
Epoch 4/16
307/307 [==============================] - 5s 15ms/step - loss: 0.2218 - acc: 0.9772 - val_loss: 2.7831 - val_acc: 0.8158
Epoch 5/16
307/307 [==============================] - 4s 15ms/step - loss: 0.0639 - acc: 0.9935 - val_loss: 2.2545 - val_acc: 0.8289
Epoch 6/16
307/307 [==============================] - 4s 14ms/step - loss: 1.6728e-05 - acc: 1.0000 - val_loss: 1.9604 - val_acc: 0.8553
Epoch 7/16
307/307 [==============================] - 5s 15ms/step - loss: 0.0410 - acc: 0.9967 - val_loss: 1.6160 - val_acc: 0.8947
Epoch 8/16
307/

In [40]:
loss, training_accuracy = model.evaluate(training_features,train_labels_onehot)

307/307 [==============================] - 1s 4ms/step


In [41]:
loss, validation_accuracy = model.evaluate(validation_features,validation_labels_onehot)

76/76 [==============================] - 0s 4ms/step


In [42]:
'Training Accuracy : '+ str(round(training_accuracy*100,1)) + '%'

'Training Accuracy : 100.0%'

In [43]:
'Validation Accuracy : '+ str(round(validation_accuracy*100,1)) + '%'

'Validation Accuracy : 86.8%'

In [44]:
resnet_results = (round(training_accuracy*100,1), round(validation_accuracy*100,1), round(features_extraction_time.total_seconds()), round(training_time.total_seconds()))

## DenseNet Evaluation

In [45]:
start_time = datetime.now()

model = applications.DenseNet201(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3)) #VGG16 trained on imagenet
training_features = images_to_feature_vectors(model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

end_time = datetime.now()
features_extraction_time = end_time - start_time
print('Features extraction time : {}'.format(features_extraction_time))

74842112/74836368 [==============================] - 5s 0us/step
Found 307 images belonging to 11 classes.
307/307 [==============================] - 109s 356ms/step
Found 76 images belonging to 11 classes.
76/76 [==============================] - 25s 332ms/step
Features extraction time : 0:03:16.499488


In [46]:
start_time = datetime.now()

model = top_model(training_features, validation_features, batch_size, epochs)

end_time = datetime.now()
training_time = end_time - start_time
print('Total training duration : {}'.format(training_time))

Train on 307 samples, validate on 76 samples
Epoch 1/16
307/307 [==============================] - 19s 61ms/step - loss: 2.6089 - acc: 0.0977 - val_loss: 1.8228 - val_acc: 0.8158
Epoch 2/16
307/307 [==============================] - 4s 14ms/step - loss: 0.2482 - acc: 0.9511 - val_loss: 1.6161 - val_acc: 0.8684
Epoch 3/16
307/307 [==============================] - 4s 14ms/step - loss: 0.0834 - acc: 0.9902 - val_loss: 1.8055 - val_acc: 0.8553
Epoch 4/16
307/307 [==============================] - 4s 14ms/step - loss: 0.0898 - acc: 0.9902 - val_loss: 1.8348 - val_acc: 0.8421
Epoch 5/16
307/307 [==============================] - 4s 14ms/step - loss: 0.0681 - acc: 0.9935 - val_loss: 1.8872 - val_acc: 0.8553
Epoch 6/16
307/307 [==============================] - 4s 14ms/step - loss: 0.0461 - acc: 0.9967 - val_loss: 2.1938 - val_acc: 0.8421
Epoch 7/16
307/307 [==============================] - 4s 13ms/step - loss: 0.0647 - acc: 0.9935 - val_loss: 2.5384 - val_acc: 0.7895
Epoch 8/16
307/307 [===

In [47]:
loss, training_accuracy = model.evaluate(training_features,train_labels_onehot)

307/307 [==============================] - 1s 3ms/step


In [48]:
loss, validation_accuracy = model.evaluate(validation_features,validation_labels_onehot)

76/76 [==============================] - 0s 4ms/step


In [49]:
'Training Accuracy : '+ str(round(training_accuracy*100,1)) + '%'

'Training Accuracy : 98.7%'

In [50]:
'Validation Accuracy : '+ str(round(validation_accuracy*100,1)) + '%'

'Validation Accuracy : 84.2%'

In [51]:
densenet_results = (round(training_accuracy*100,1), round(validation_accuracy*100,1), round(features_extraction_time.total_seconds()), round(training_time.total_seconds()))

In [52]:
results = {'Model': ['VGG16', 'Inception', 'Resnet', 'Densnet'], 
           'Training Accuracy(%)': [vgg16_results[0], inception_results[0], resnet_results[0], densenet_results[0]],
           'Validation Accuracy(%)': [vgg16_results[1], inception_results[1], resnet_results[1], densenet_results[1]],
           'Features Extraction time(s)': [vgg16_results[2], inception_results[2], resnet_results[2], densenet_results[2]],
           'Training time(s)': [vgg16_results[3], inception_results[3], resnet_results[3], densenet_results[3]]}
df = pd.DataFrame(data=results)

In [53]:
df

,Model,Training Accuracy(%),Validation Accuracy(%),Features Extraction time(s),Training time(s)
0,VGG16,100.0,80.3,99,22
1,Inception,88.9,59.2,78,43
2,Resnet,100.0,86.8,90,82
3,Densnet,98.7,84.2,196,83
